In [1]:
import keras
from keras.layers import Input
import sys
import os
sys.path.insert(0, "../models")
sys.path.insert(0, "../layers")
import models

Using TensorFlow backend.
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

### Load the trained Float model, and untrained BNN

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
import yaml
from train import *

In [6]:
class dummy:
    def __init__(self):
        self.x = 0
options = dummy()
options.inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z'
options.outputDir = 'binary_model_float_initialised'
os.mkdir(options.outputDir)
options.tree = 't_allpar_new'
yamlConfig = parse_config('train_config_threelayer_binary.yml')
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Loading configuration from train_config_threelayer_binary.yml


/Users/sionisummers/Work/hls4ml/keras-training-clean/train/train.py:137: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(config)


(986806,)
('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_undef')


In [7]:
float_model = keras.models.load_model("float_model/KERAS_check_best_model.h5")
binary_model = models.three_layer_model_binary_relu(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'])
ifc = 0
for il in range(len(float_model.layers) - 1):
    if 'fc' in float_model.layers[il+1].name:
        # Get the weights (ignore the biases)
        w = float_model.layers[il+1].get_weights()[0]
        binary_model.layers[il].set_weights([w])
keras_model = binary_model
print_model_to_json(keras_model,options.outputDir + '/' + 'KERAS_model.json')

In [8]:
startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])

In [9]:
callbacks=all_callbacks(stop_patience=1000,
                         lr_factor=0.5,
                         lr_patience=10,
                         lr_epsilon=0.000001,
                         lr_cooldown=2,
                         lr_minimum=0.0000001,
                         outputDir=options.outputDir)

/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
 keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 1,
                 validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

Train on 592083 samples, validate on 197361 samples
Epoch 1/100
592083/592083 [==============================] - 5s 9us/step - loss: 0.8870 - accuracy: 0.6308 - val_loss: 0.8579 - val_accuracy: 0.5417

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00001: val_loss improved from inf to 0.85785, saving model to binary_model_float_initialised/KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 0.85785, saving model to binary_model_float_initialised/KERAS_check_best_model_weights.h5

Epoch 00001: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00001: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 2/100
592083/592083 [==============================] - 5s 9us/step - loss: 0.7365 - accuracy: 0.6613 - val_loss: 0.8015 - val_accuracy: 0.6391

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00002: val_loss improved fr

592083/592083 [==============================] - 4s 7us/step - loss: 0.3643 - accuracy: 0.6915 - val_loss: 0.4613 - val_accuracy: 0.6221

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00014: val_loss did not improve from 0.41758

Epoch 00014: val_loss did not improve from 0.41758

Epoch 00014: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00014: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 15/100
592083/592083 [==============================] - 5s 8us/step - loss: 0.3628 - accuracy: 0.6907 - val_loss: 0.4552 - val_accuracy: 0.5662

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00015: val_loss did not improve from 0.41758

Epoch 00015: val_loss did not improve from 0.41758

Epoch 00015: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00015: saving model to binary_model_float_initialised/K


Epoch 00028: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

***callbacks end***

Epoch 29/100
592083/592083 [==============================] - 4s 7us/step - loss: 0.3511 - accuracy: 0.6977 - val_loss: 0.5960 - val_accuracy: 0.4260

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00029: val_loss did not improve from 0.40939

Epoch 00029: val_loss did not improve from 0.40939

Epoch 00029: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00029: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 30/100
592083/592083 [==============================] - 4s 7us/step - loss: 0.3525 - accuracy: 0.6967 - val_loss: 0.3919 - val_accuracy: 0.6600

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00030: val_loss improved from 0.40939 to 0.39194, saving model to binary_model_float_initialised/KERAS_check_best_model.h5

Epoch 

Epoch 44/100
592083/592083 [==============================] - 4s 7us/step - loss: 0.3518 - accuracy: 0.6968 - val_loss: 0.4831 - val_accuracy: 0.5217

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00044: val_loss did not improve from 0.37864

Epoch 00044: val_loss did not improve from 0.37864

Epoch 00044: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00044: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 45/100
592083/592083 [==============================] - 4s 7us/step - loss: 0.3506 - accuracy: 0.6977 - val_loss: 0.4784 - val_accuracy: 0.5583

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00045: val_loss did not improve from 0.37864

Epoch 00045: val_loss did not improve from 0.37864

Epoch 00045: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00045: saving model to binary_model_float_

592083/592083 [==============================] - 4s 7us/step - loss: 0.3510 - accuracy: 0.6973 - val_loss: 0.4127 - val_accuracy: 0.6304

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00059: val_loss did not improve from 0.37864

Epoch 00059: val_loss did not improve from 0.37864

Epoch 00059: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00059: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 60/100
592083/592083 [==============================] - 5s 8us/step - loss: 0.3514 - accuracy: 0.6969 - val_loss: 0.4333 - val_accuracy: 0.6284

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00060: val_loss did not improve from 0.37864

Epoch 00060: val_loss did not improve from 0.37864

Epoch 00060: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00060: saving model to binary_model_float_initialised/K

592083/592083 [==============================] - 5s 9us/step - loss: 0.3514 - accuracy: 0.6975 - val_loss: 0.5236 - val_accuracy: 0.5192

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00074: val_loss did not improve from 0.37864

Epoch 00074: val_loss did not improve from 0.37864

Epoch 00074: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00074: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 75/100
592083/592083 [==============================] - 6s 10us/step - loss: 0.3498 - accuracy: 0.6990 - val_loss: 0.5079 - val_accuracy: 0.5415

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00075: val_loss did not improve from 0.37864

Epoch 00075: val_loss did not improve from 0.37864

Epoch 00075: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00075: saving model to binary_model_float_initialised/

592083/592083 [==============================] - 5s 8us/step - loss: 0.3504 - accuracy: 0.6980 - val_loss: 0.4739 - val_accuracy: 0.5668

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00089: val_loss did not improve from 0.37864

Epoch 00089: val_loss did not improve from 0.37864

Epoch 00089: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00089: saving model to binary_model_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 90/100
592083/592083 [==============================] - 5s 8us/step - loss: 0.3497 - accuracy: 0.6981 - val_loss: 0.4504 - val_accuracy: 0.5817

***callbacks***
saving losses to binary_model_float_initialised/losses.log

Epoch 00090: val_loss did not improve from 0.37864

Epoch 00090: val_loss did not improve from 0.37864

Epoch 00090: saving model to binary_model_float_initialised/KERAS_check_model_last.h5

Epoch 00090: saving model to binary_model_float_initialised/K